# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
read_file = "../Data/cities.csv"
DF = pd.read_csv(read_file)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
gmap = gmaps.figure(layout=layout)

In [5]:
locs = DF[['Lat', 'Lng']].astype(float)
intense = DF['Humidity'].max()
heat_layer = gmaps.heatmap_layer(locs, weights=DF['Humidity'], 
                                 dissipating=False, max_intensity=intense,
                                 point_radius = 1)
gmap.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowDFa = DF[DF['Max Temp'] > 50]
narrowDFb = DF[DF['Max Temp'] < 90]
narrowDF = narrowDFa.merge(narrowDFb)
bestDFa = narrowDF[narrowDF['Humidity'] < 60]
bestDFb = bestDFa[bestDFa['Wind Speed'] < 5]
bestDF = bestDFb[bestDFb['Cloudiness'] < 70]
bestDF.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Ushuaia,-54.8000,-68.3000,57.20,44,40,3.44,AR,1614803199
5,Saint George,37.1041,-113.5841,69.01,11,1,4.61,US,1614803301
108,Basoko,1.2391,23.6160,78.19,45,7,3.31,CD,1614803368
115,Dābhol,17.6000,73.1667,75.11,47,0,3.96,IN,1614803370
140,Bāfq,31.6128,55.4107,58.46,32,0,1.90,IR,1614803376
151,Pokhara,28.2333,83.9833,57.61,34,0,4.76,NP,1614803381
156,Buta,2.7858,24.7300,77.31,30,0,1.36,CD,1614803382
161,Baripāda,21.9350,86.7214,68.13,25,0,3.29,IN,1614803383
179,Portland,45.5234,-122.6762,55.99,57,20,2.42,US,1614802935
195,Arāria,26.1500,87.5167,62.40,28,0,2.95,IN,1614803394


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = bestDF
hotel_df['Hotel Name'] = ""

<ipython-input-7-1605f0671867>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


In [8]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for i, r in hotel_df.iterrows():
    params['location'] = f"{r['Lat']}, {r['Lng']}"
    response = requests.get(url, params).json()
    try:
        print(f"Retrieving Results for nearest hotel in {r['City']}.")
        hotel_df.loc[i, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print('No results found...')

Retrieving Results for nearest hotel in Ushuaia.


C:\Users\ADMIN\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Retrieving Results for nearest hotel in Saint George.
Retrieving Results for nearest hotel in Basoko.
Retrieving Results for nearest hotel in Dābhol.
Retrieving Results for nearest hotel in Bāfq.
Retrieving Results for nearest hotel in Pokhara.
Retrieving Results for nearest hotel in Buta.
Retrieving Results for nearest hotel in Baripāda.
Retrieving Results for nearest hotel in Portland.
Retrieving Results for nearest hotel in Arāria.
Retrieving Results for nearest hotel in Pedernales.
Retrieving Results for nearest hotel in Rajpur.
Retrieving Results for nearest hotel in Bryan.
Retrieving Results for nearest hotel in Clocolan.
Retrieving Results for nearest hotel in Sakakah.
Retrieving Results for nearest hotel in Lokoja.
Retrieving Results for nearest hotel in Hope.
Retrieving Results for nearest hotel in Magway.
Retrieving Results for nearest hotel in Malakanagiri.
Retrieving Results for nearest hotel in Keïta.
No results found...
Retrieving Results for nearest hotel in Ranfurly.
Re

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
gmap.add_layer(marker_layer)

# Display figure
gmap

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…